In [ ]:
from google.colab import drive
from google.colab import files
import os
import glob
import pickle 
import torch
import shutil

In [ ]:
# Rescata los datos guardados previamente.
# Ejecuta esta celda una vez al principio de tu sesión.
drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2

print(" Autoreload activado. Los archivos .py se recargarán automáticamente.")

Mounted at /content/drive
✅ Autoreload activado. Los archivos .py se recargarán automáticamente.


In [ ]:
# Define la ruta de tu proyecto en Google Drive
project_path_drive = '/content/drive/MyDrive/SQHN_Project_Results'
os.makedirs(project_path_drive, exist_ok=True)
print(f"Carpeta de proyecto en Drive asegurada en: {project_path_drive}")

# Define la ruta donde se guardan los resultados en Colab
source_folder_colab = '/content/SQHN/results' # O '/content/SQHN/data' si los .data se guardan allí

# Define la carpeta de destino para esta ejecución específica
destination_folder_drive = os.path.join(project_path_drive, 'OnCont_L1_Results')

# El comando de copia (no lo ejecutes todavía, solo déjalo listo)
# Usamos -r para copiar el directorio completo y -v para ver qué se copia
copy_command = f"cp -rv {source_folder_colab} {destination_folder_drive}"

print("\nEl comando para guardar los resultados está listo.")
print(f"Se ejecutará: {copy_command}")

Carpeta de proyecto en Drive asegurada en: /content/drive/MyDrive/SQHN_Project_Results

El comando para guardar los resultados está listo.
Se ejecutará: cp -rv /content/SQHN/results /content/drive/MyDrive/SQHN_Project_Results/OnCont_L1_Results


In [ ]:
# Celda 0: Verificar la GPU
!nvidia-smi

Mon Jul 21 04:45:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Celda 1: Clonar el repositorio de SQHN
print("Clonando el repositorio de SQHN...")
!git clone https://github.com/nalonso2/SQHN.git
print("Repositorio clonado.")

# Movernos al directorio del proyecto para que los siguientes comandos se ejecuten allí
%cd SQHN

Clonando el repositorio de SQHN...
Cloning into 'SQHN'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 84 (delta 0), reused 0 (delta 0), pack-reused 81 (from 1)
Receiving objects: 100% (84/84), 63.20 KiB | 15.80 MiB/s, done.
Resolving deltas: 100% (48/48), done.
Repositorio clonado.
/content/SQHN


In [ ]:
# Celda 2: Instalar la versión específica de PyTorch
# Comando corregido para instalar una versión moderna de PyTorch
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

print("Instalación de PyTorch (versión moderna) completada.")

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.7 MB/s e

In [ ]:
# Celda 3: Descargar y preparar Tiny ImageNet
# Crear el directorio 'data' si no existe
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Directorio '{data_dir}' creado.")

# Descargar el archivo zip
print("Descargando Tiny ImageNet (esto puede tardar unos minutos)...")
!wget -q http://cs231n.stanford.edu/tiny-imagenet-200.zip -P {data_dir}
print("Descarga completada.")

# Descomprimir el archivo en el directorio 'data'
print("Descomprimiendo el archivo...")
!unzip -q {data_dir}/tiny-imagenet-200.zip -d {data_dir}
print("Descompresión completada.")

Descargando Tiny ImageNet (esto puede tardar unos minutos)...
Descarga completada.
Descomprimiendo el archivo...
Descompresión completada.
total 2612
drwxrwxr-x   3 root root    4096 Dec 12  2014 test
drwxrwxr-x 202 root root    4096 Dec 12  2014 train
drwxrwxr-x   3 root root    4096 Dec 12  2014 val
-rw-rw-r--   1 root root    2000 Feb  9  2015 wnids.txt
-rw-------   1 root root 2655750 Feb  9  2015 words.txt


In [ ]:
# Celda 3.1: Corregir el código para que descargue los datasets automáticamente
# Lista de archivos a modificar
files_to_patch = glob.glob('**/*.py', recursive=True)

print("Archivos que se van a revisar para parcheo:")
for f in files_to_patch:
    print(f"- {f}")

# --- PARCHE 1: Habilitar descargas automáticas ---
print("\n[Parche 1] Habilitando 'download=True'...")
for file_path in files_to_patch:
    !sed -i 's/download=False/download=True/g' {file_path}

# --- PARCHE 2: Corregir la ruta 'hardcoded' de Tiny ImageNet ---
# La ruta original del autor
hardcoded_path = 'C:/Users/nalon/Documents/PythonScripts/DataSets/tiny-imagenet-200'
# La ruta relativa correcta que hemos creado en Colab
correct_path = './data/tiny-imagenet-200'

print(f"\n[Parche 2] Reemplazando ruta absoluta '{hardcoded_path}' por ruta relativa '{correct_path}'...")
for file_path in files_to_patch:
    # Usamos un delimitador diferente (#) en sed porque las rutas contienen '/'
    # Esto evita tener que escapar cada barra en la ruta
    !sed -i 's#{hardcoded_path}#{correct_path}#g' {file_path}

print("\nParcheo completado.")

Archivos que se van a revisar para parcheo:
- plot_image_grid.py
- emerge_anylz.py
- noisy_aa_onlineBP.py
- show_reconstruct.py
- MHN_Tree.py
- aa_online_tree_pre.py
- noise_encode_img.py
- recognition_MHN.py
- recognition.py
- noisy_aa_online.py
- noisy_aa_onlineBPTree.py
- MHN_layer.py
- infer_compare.py
- main.py
- aa_online.py
- noisy_aa_onlineTree.py
- utilities.py
- LocLayer.py
- MHN_EWC.py
- plot.py
- recognition_tree.py
- autoassociate.py
- heteroassociate.py
- tree_energy_test.py
- MHN_EWC_Tree.py
- trainer.py
- MHN.py
- aa_online_tree.py
- plot_images.py
- Tree.py
- aa_online_BPtree.py
- aa_onlineBP.py
- Unit.py

[Parche 1] Habilitando 'download=True'...

[Parche 2] Reemplazando ruta absoluta 'C:/Users/nalon/Documents/PythonScripts/DataSets/tiny-imagenet-200' por ruta relativa './data/tiny-imagenet-200'...

Parcheo completado.


In [ ]:
# Sincronizar Drive -> Colab
print("Restaurando resultados previos desde Google Drive...")

# Define las rutas
project_path_drive = '/content/drive/MyDrive/SQHN_Project_Final'
results_source_drive = os.path.join(project_path_drive, 'OnCont_L1_Results')
results_dest_colab = '/content/SQHN/results' # Directorio de checkpoints

# Asegurarse de que los directorios existen
!mkdir -p {results_source_drive}
!mkdir -p {results_dest_colab}

# Copiar los archivos .data existentes desde Drive a Colab
# -v: verbose, -n: no-clobber (no sobrescribir si ya existe)
!cp -vn {results_source_drive}/*.data {results_dest_colab}/

print("Sincronización inicial completada.")

Restaurando resultados previos desde Google Drive...
'/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Results/AA_OnlineBP_data4_opt0_hdsz2000.data' -> '/content/SQHN/results/AA_OnlineBP_data4_opt0_hdsz2000.data'
'/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Results/AA_OnlineBP_data4_opt0_hdsz3300.data' -> '/content/SQHN/results/AA_OnlineBP_data4_opt0_hdsz3300.data'
'/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Results/AA_OnlineBP_data4_opt0_hdsz700.data' -> '/content/SQHN/results/AA_OnlineBP_data4_opt0_hdsz700.data'
'/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Results/AA_OnlineBP_data4_opt1_hdsz2000.data' -> '/content/SQHN/results/AA_OnlineBP_data4_opt1_hdsz2000.data'
'/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Results/AA_OnlineBP_data4_opt1_hdsz3300.data' -> '/content/SQHN/results/AA_OnlineBP_data4_opt1_hdsz3300.data'
'/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Results/AA_OnlineBP_data4_opt1_hdsz700.data' -> '/content/SQHN/results/AA_OnlineB

In [ ]:
# Prepárate, este tardará más que el anterior.
print("Ejecutando el test de aprendizaje online-continual (OnCont-L1)...")
# ¡Fíjate cómo pasamos la variable 'destination_drive' al comando!
!python main.py --test OnCont-L1 --drive_path {destination_drive}

# EN CUANTO LA CELDA 4 TERMINE, EJECUTA ESTA.
print("Guardando los resultados de 'OnCont-L1' en Google Drive...")

# Definimos las rutas de nuevo para estar seguros
project_path_drive = '/content/drive/MyDrive/SQHN_Project_Final'
results_source_colab = '/content/SQHN/results'
destination_drive = os.path.join(project_path_drive, 'OnCont_L1_Results')

# Copiar la carpeta de resultados
!cp -rv {results_source_colab} {destination_drive}

print(f"¡Éxito! Resultados guardados en {destination_drive}")

Ejecutando el test de aprendizaje online-continual (OnCont-L1)...
--- Iniciando bloque de tests: online_L1 (Online Learning) ---

[Sub-bloque 1/3] EMNIST
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_Online_Simf2_numN300_data6_numData3000_upType0_det0.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt0_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt1_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt2_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt3_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_Online_Simf2_numN1300_data6_numData3000_upType0_det0.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt0_hdsz1300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt1_hdsz1300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_O

In [ ]:
#### **CELDA 6: Generar los Gráficos (Ahora o en una sesión futura)**```python
# Ahora que los resultados están a salvo, puedes generar los gráficos.
# Si la sesión se desconecta, en la próxima sesión puedes saltar directamente
# a esta celda después de montar Drive y restaurar los resultados.

print("Generando los gráficos para 'OnCont-L1'...")
!python main.py --plot OnCont-L1

# Copiar también los gráficos generados a Drive
plots_source_colab = '/content/SQHN/plots' # Asumiendo que se guardan aquí
destination_plots_drive = os.path.join(project_path_drive, 'OnCont_L1_Plots')
!cp -rv {plots_source_colab} {destination_plots_drive}

print(f"Gráficos guardados en {destination_plots_drive}")

Generando los gráficos para 'OnCont-L1'...
✅ Gráfico guardado localmente en: plots/OnCont_L1_Sensitivity.png
Figure(600x150)
/content/SQHN/plot.py:462: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  axs[0,d].errorbar(x, torch.mean(dta[3], dim=0), yerr=torch.std(dta[3], dim=0), fmt='o', alpha=.6, ls='-', label=names[o], markersize=2.5, color=clrs[o], marker='s')
/content/SQHN/plot.py:463: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  axs[1,d].errorbar(x, torch.mean(dta[2], dim=0), yerr=torch.std(dta[2], dim=0), fmt='o', alpha=.6, ls='-', label=names[o], markersize=2.5, color=clrs[o], marker='s')
/content/SQHN/plot.py:478: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will 

In [ ]:
%load_ext autoreload
%autoreload 2

print("✅ Autoreload activado. Los archivos .py se recargarán automáticamente.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✅ Autoreload activado. Los archivos .py se recargarán automáticamente.


In [ ]:
print("🚀 Iniciando el test de aprendizaje 'OnCont-L1'...")
print("   El script saltará los trabajos completados y ejecutará los que faltan.")
print("   Cada resultado se guardará en Drive de forma inmediata.")

!python main.py --test OnCont-L1 --drive_path {destination_drive_results}

print("\n✅ ¡Todos los entrenamientos del test 'OnCont-L1' han finalizado!")

🚀 Iniciando el test de aprendizaje 'OnCont-L1'...
   El script saltará los trabajos completados y ejecutará los que faltan.
   Cada resultado se guardará en Drive de forma inmediata.
--- Iniciando bloque de tests: online_L1 (Online Learning) ---

[Sub-bloque 1/3] EMNIST
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_Online_Simf2_numN300_data6_numData3000_upType0_det0.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt0_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt1_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt2_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt3_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AE_Online_data6_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_Online_Simf2_numN1300_data6_numData3000_upType0_det0.data'
✔️ SALTANDO: El archivo ya existe en Colab 

In [ ]:
# --- CELDA FINAL: Generar y Guardar los Gráficos (Versión Robusta) ---
# 1. Redefinir las rutas clave para asegurarnos de que existen en esta celda
project_path_drive = '/content/drive/MyDrive/SQHN_Project_Final'
destination_drive_plots = os.path.join(project_path_drive, 'OnCont_L1_Plots')
plots_source_colab = '/content/SQHN/plots'

# 2. Asegurarse de que la carpeta de destino en Drive existe
os.makedirs(destination_drive_plots, exist_ok=True)

# 3. Generar los gráficos (esto ya sabes que funciona)
print("📊 Generando los gráficos para 'OnCont-L1'...")
!python main.py --plot OnCont-L1

# 4. Copiar los archivos a Drive usando f-string para garantizar la correcta
#    sustitución de las variables.
print("\nCopiando los gráficos generados a Google Drive...")
!cp -rv {plots_source_colab}/* {destination_drive_plots}/

print(f"\n✅ ¡ÉXITO! Gráficos guardados en: {destination_drive_plots}")

📊 Generando los gráficos para 'OnCont-L1'...
✅ Gráfico guardado localmente en: plots/OnCont_L1_Sensitivity.png
Figure(600x150)
/content/SQHN/plot.py:462: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  axs[0,d].errorbar(x, torch.mean(dta[3], dim=0), yerr=torch.std(dta[3], dim=0), fmt='o', alpha=.6, ls='-', label=names[o], markersize=2.5, color=clrs[o], marker='s')
/content/SQHN/plot.py:463: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  axs[1,d].errorbar(x, torch.mean(dta[2], dim=0), yerr=torch.std(dta[2], dim=0), fmt='o', alpha=.6, ls='-', label=names[o], markersize=2.5, color=clrs[o], marker='s')
/content/SQHN/plot.py:478: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument wil

In [ ]:
# --- CONFIGURA ESTOS PARÁMETROS ---
# Cambia el nombre del archivo para ver los resultados del modelo que quieras.

# Ejemplo 1: SQHN (el más grande de EMNIST)
filename_sqhn = 'results/AA_OnlineCont_Simf2_numN2300_data6_numData3000_upType0_det0.data'

# Ejemplo 2: Autoencoder (el más grande de EMNIST)
filename_ae = 'results/AE_Online_data6_hdsz2300.data'
# ------------------------------------


def inspect_results(filename):
    """
    Carga un archivo de resultados y muestra los valores finales de MSE y Acc.
    """
    print(f"--- Inspeccionando el archivo: {filename} ---")
    try:
        with open(filename, 'rb') as filehandle:
            data = pickle.load(filehandle)

        # En nuestros archivos .data:
        # data[2] es el tensor de recall_mse_n
        # data[3] es el tensor de recall_pcnt_n

        mse_tensor = data[2]
        acc_tensor = data[3]

        # Calculamos la media del rendimiento final (la última columna de cada tensor)
        final_mse_mean = torch.mean(mse_tensor[:, -1])
        final_mse_std = torch.std(mse_tensor[:, -1])

        final_acc_mean = torch.mean(acc_tensor[:, -1])
        final_acc_std = torch.std(acc_tensor[:, -1])

        print(f"  -> MSE Final Promedio: {final_mse_mean.item():.6f} (std: {final_mse_std.item():.6f})")
        print(f"  -> Acc Final Promedio: {final_acc_mean.item():.6f} (std: {final_acc_std.item():.6f})")
        print("-" * 40 + "\n")

    except FileNotFoundError:
        print(f"  -> ERROR: No se encontró el archivo. Verifica el nombre.")
    except Exception as e:
        print(f"  -> Ocurrió un error: {e}")

# Ejecutamos la inspección
inspect_results(filename_sqhn)
inspect_results(filename_ae)

--- Inspeccionando el archivo: results/AA_OnlineCont_Simf2_numN2300_data6_numData3000_upType0_det0.data ---
  -> MSE Final Promedio: 0.004538 (std: 0.000126)
  -> Acc Final Promedio: 0.809327 (std: 0.007265)
----------------------------------------

--- Inspeccionando el archivo: results/AE_Online_data6_hdsz2300.data ---
  -> MSE Final Promedio: 0.000000 (std: 0.000000)
  -> Acc Final Promedio: 0.000000 (std: 0.000000)
----------------------------------------



In [ ]:
# --- CELDA 7: Visualizar y Guardar Reconstrucciones ---
# 1. Redefinir la ruta de destino de los gráficos
project_path_drive = '/content/drive/MyDrive/SQHN_Project_Final'
destination_drive_plots = os.path.join(project_path_drive, 'OnCont_L1_Plots')
plots_source_colab = '/content/SQHN/plots'

# 2. Ejecutar el script de visualización
#    ¡Puedes cambiar los parámetros aquí!
#    --num_images: Cuántas imágenes mostrar.
#    --dataset_id: 6=EMNIST, 0=MNIST, 1=FashionMNIST.
#    --epochs: Cuánto entrenar el autoencoder (más épocas = mejor reconstrucción).
print("🖼️  Generando y guardando la comparación visual...")
!python visualize.py --num_images 8 --dataset_id 0 --epochs 200

# 3. Copiar la imagen generada a tu carpeta de gráficos en Drive
print("\nCopiando la visualización a Google Drive...")
!cp -rv {plots_source_colab}/reconstruction_*.png {destination_drive_plots}/

print(f"\n✅ ¡Éxito! Visualización guardada en: {destination_drive_plots}")

🖼️  Generando y guardando la comparación visual...
Cargando 8 imágenes del dataset ID: 0...
Entrenando SQHN...
Entrenando Autoencoder por 200 épocas...
Generando visualización...
✅ Visualización guardada localmente en: plots/reconstruction_comparison_data0_8imgs.png
Figure(600x1600)

Copiando la visualización a Google Drive...
'/content/SQHN/plots/reconstruction_comparison_data0_8imgs.png' -> '/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Plots/reconstruction_comparison_data0_8imgs.png'

✅ ¡Éxito! Visualización guardada en: /content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Plots


In [ ]:
# --- CELDA DE VISUALIZACIÓN 1: TEST DE LOTE ---
print("🖼️  Generando comparación en Lote (Test de Generalización)...")
# Este script muestra cómo el Autoencoder generaliza bien, mientras que el
# SQHN sufre de interferencia catastrófica al memorizar todo a la vez.
!python visualize_batch.py --num_images 8 --dataset_id 0 --epochs 200

# Copiar la imagen generada a Drive
plots_source_colab = '/content/SQHN/plots'
# ... (código para copiar el archivo ..._batch_...) ...
print("\nCopiando visualización de Lote a Google Drive...")
!cp -rv {plots_source_colab}/reconstruction_batch_*.png {destination_drive_plots}/

🖼️  Generando comparación en Lote (Test de Generalización)...
Cargando 8 imágenes del dataset ID: 0...
Entrenando SQHN con todo el lote (demostrando interferencia)...
Entrenando Autoencoder por 200 épocas...
Generando visualización de Lote...
✅ Visualización guardada localmente en: plots/reconstruction_batch_comparison_data0_8imgs.png
Figure(600x1600)

Copiando visualización de Lote a Google Drive...
'/content/SQHN/plots/reconstruction_batch_comparison_data0_8imgs.png' -> '/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Plots/reconstruction_batch_comparison_data0_8imgs.png'


In [ ]:
# --- CELDA DE VISUALIZACIÓN 2: TEST ONE-SHOT ---
print("🖼️  Generando comparación One-Shot (Test de Memoria Perfecta)...")
# Este script muestra cómo el SQHN puede memorizar y reconstruir una
# única imagen a la perfección, mostrando su capacidad de recuerdo.
!python visualize_oneshot.py --num_images 8 --dataset_id 0 --epochs 200

# Copiar la imagen generada a Drive
plots_source_colab = '/content/SQHN/plots'
# ... (código para copiar el archivo ..._oneshot_...) ...
print("\nCopiando visualización One-Shot a Google Drive...")
!cp -rv {plots_source_colab}/reconstruction_oneshot_*.png {destination_drive_plots}/

🖼️  Generando comparación One-Shot (Test de Memoria Perfecta)...
Cargando 8 imágenes del dataset ID: 0...
Generando reconstrucciones de SQHN (una memoria por imagen)...
Entrenando Autoencoder por 200 épocas...
Generando visualización One-Shot...
✅ Visualización guardada localmente en: plots/reconstruction_oneshot_comparison_data0_8imgs.png
Figure(600x1600)

Copiando visualización One-Shot a Google Drive...
'/content/SQHN/plots/reconstruction_oneshot_comparison_data0_8imgs.png' -> '/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Plots/reconstruction_oneshot_comparison_data0_8imgs.png'


In [ ]:
# --- CELDA FINAL: Ejecutar la Demostración de Restauración ---
# 1. Definimos los nombres de los archivos que subiste
original_filename = "logo_original.png"
corrupted_filename = "logo_corrupto.png"

print(f"🖼️  Iniciando la demostración de restauración...")
print(f"   - Modelo aprenderá de: '{original_filename}'")
print(f"   - Modelo intentará restaurar: '{corrupted_filename}'")

# 2. Ejecutar el script de visualización
#    Le pasamos los dos nombres de archivo como argumentos.
!python visualize_custom.py {original_filename} {corrupted_filename}

# 3. Copiar la imagen final generada a Google Drive
#    (Redefinimos las rutas aquí para que la celda sea independiente)
project_path_drive = '/content/drive/MyDrive/SQHN_Project_Final'
destination_drive_plots = os.path.join(project_path_drive, 'OnCont_L1_Plots')
plots_source_colab = '/content/SQHN/plots'

print("\nCopiando la imagen final a Google Drive...")
# El nombre del archivo de salida está definido dentro de visualize_custom.py
output_filename = "custom_image_restoration.png"
!cp -v {os.path.join(plots_source_colab, output_filename)} {destination_drive_plots}/

print(f"\n✅ ¡Éxito! Visualización guardada en: {destination_drive_plots}")

🖼️  Iniciando la demostración de restauración...
   - Modelo aprenderá de: 'logo_original.png'
   - Modelo intentará restaurar: 'logo_corrupto.png'
Imágenes cargadas. Tamaño del vector de entrada: 3072
Enseñando la imagen ORIGINAL al SQHN (memoria one-shot)...
Restaurando la imagen CORRUPTA con SQHN...
Entrenando Autoencoder con la imagen ORIGINAL por 300 épocas...
Restaurando la imagen CORRUPTA con Autoencoder...
Generando comparación final...
✅ Visualización guardada localmente en: plots/custom_image_restoration.png
Figure(1200x400)

Copiando la imagen final a Google Drive...
'/content/SQHN/plots/custom_image_restoration.png' -> '/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Plots/custom_image_restoration.png'

✅ ¡Éxito! Visualización guardada en: /content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Plots


In [ ]:
print(f"🖼️  Iniciando test de oclusión severa en MNIST...")
!python visualize_mnist_occlusion.py

# Copiar el resultado a Drive
project_path_drive = '/content/drive/MyDrive/SQHN_Project_Final'
destination_drive_plots = os.path.join(project_path_drive, 'OnCont_L1_Plots')
plots_source_colab = '/content/SQHN/plots'
output_filename = "mnist_occlusion_test.png"

print("\nCopiando el resultado a Google Drive...")
!cp -v {os.path.join(plots_source_colab, output_filename)} {destination_drive_plots}/
print(f"\n✅ ¡Éxito! Visualización guardada en: {destination_drive_plots}")

🖼️  Iniciando test de oclusión severa en MNIST...
Imagen de MNIST cargada. Tamaño del vector: 784
Ocultando la mitad derecha de la imagen...
Enseñando el dígito ORIGINAL al SQHN...
Restaurando el dígito OCULTO con SQHN...
Entrenando Autoencoder con el dígito ORIGINAL por 200 épocas...
Restaurando el dígito OCULTO con Autoencoder...
Generando comparación final...
✅ Visualización guardada localmente en: plots/mnist_occlusion_test.png
Figure(1200x400)

Copiando el resultado a Google Drive...
'/content/SQHN/plots/mnist_occlusion_test.png' -> '/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Plots/mnist_occlusion_test.png'

✅ ¡Éxito! Visualización guardada en: /content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Plots


In [ ]:
# --- PASO 1: Preparar el Entorno ---
# Crear las carpetas para las imágenes limpias y las corruptas si no existen.
clean_folder = "qr_codes_clean"
corrupted_folder = "qr_codes_corrupted"
os.makedirs(clean_folder, exist_ok=True)
os.makedirs(corrupted_folder, exist_ok=True)

print("✅ Carpetas 'qr_codes_clean' y 'qr_codes_corrupted' listas.")
print("-" * 50)

# --- PASO 2: Subir las Imágenes de Referencia (Limpias) ---
print("Por favor, sube tus imágenes de QR originales y limpias:")
print("(Puedes seleccionar varios archivos a la vez)")

# Mover los archivos subidos a la carpeta correcta
uploaded_clean = files.upload()
if not uploaded_clean:
    print("\n⚠️ No se subieron archivos limpios. El proceso se detendrá.")
else:
    for filename, content in uploaded_clean.items():
        shutil.move(filename, os.path.join(clean_folder, filename))
    print(f"\n✅ {len(uploaded_clean)} imágenes limpias movidas a la carpeta '{clean_folder}'.")
    print("-" * 50)

    # --- PASO 3: Construir la Biblioteca de Modelos ---
    print("\n🧠 Construyendo la biblioteca de modelos a partir de las imágenes limpias...")
    print("(Esto puede tardar unos minutos, ya que se entrena un modelo para cada imagen)")
    !python qr_restorer.py learn {clean_folder}
    print("-" * 50)

    # --- PASO 4: Subir la Imagen Corrupta a Restaurar ---
    print("\nAhora, por favor, sube la imagen de QR corrupta que quieres restaurar:")
    print("(Selecciona solo un archivo)")

    uploaded_corrupted = files.upload()
    if not uploaded_corrupted:
        print("\n⚠️ No se subió ningún archivo corrupto. El proceso se detendrá.")
    else:
        corrupted_filename = list(uploaded_corrupted.keys())[0]
        shutil.move(corrupted_filename, os.path.join(corrupted_folder, corrupted_filename))
        image_to_restore_path = os.path.join(corrupted_folder, corrupted_filename)
        print(f"\n✅ Imagen corrupta '{corrupted_filename}' lista para ser restaurada.")
        print("-" * 50)

        # --- PASO 5: Restaurar la Imagen ---
        print(f"\n🖼️  Iniciando la restauración de '{corrupted_filename}'...")
        !python qr_restorer.py restore {image_to_restore_path}

        # --- PASO 6: Guardar el Resultado en Google Drive ---
        project_path_drive = '/content/drive/MyDrive/SQHN_Project_Final'
        destination_drive_plots = os.path.join(project_path_drive, 'Restauraciones_QR')
        os.makedirs(destination_drive_plots, exist_ok=True)

        plots_source_colab = '/content/SQHN/plots'
        output_filename = "qr_restoration_result.png"
        source_file_path = os.path.join(plots_source_colab, output_filename)

        if os.path.exists(source_file_path):
            print("\nCopiando la imagen final a Google Drive...")
            !cp -v {source_file_path} {destination_drive_plots}/
            print(f"\n✅ ¡ÉXITO! Visualización guardada en: {destination_drive_plots}")
        else:
            print("\n⚠️ No se encontró el archivo de resultado para copiar a Drive.")

✅ Carpetas 'qr_codes_clean' y 'qr_codes_corrupted' listas.
--------------------------------------------------
Por favor, sube tus imágenes de QR originales y limpias:
(Puedes seleccionar varios archivos a la vez)


Saving qr_lote_a.png to qr_lote_a.png
Saving qr_lote_b.png to qr_lote_b.png
Saving qr_lote_c.png to qr_lote_c.png
Saving qr_lote_d.png to qr_lote_d.png
Saving qr_lote_e.png to qr_lote_e.png
Saving qr_lote_f.png to qr_lote_f.png
Saving qr_lote_g.png to qr_lote_g.png
Saving qr_lote_h.png to qr_lote_h.png
Saving qr_lote_j.png to qr_lote_j.png
Saving qr_lote_k.png to qr_lote_k.png

✅ 10 imágenes limpias movidas a la carpeta 'qr_codes_clean'.
--------------------------------------------------

🧠 Construyendo la biblioteca de modelos a partir de las imágenes limpias...
(Esto puede tardar unos minutos, ya que se entrena un modelo para cada imagen)
Construyendo biblioteca de modelos desde: qr_codes_clean
  -> Entrenando modelos para 'qr_lote_d.png'...
  -> Entrenando modelos para 'qr_lote_f.png'...
  -> Entrenando modelos para 'qr_lote_g.png'...
  -> Entrenando modelos para 'qr_lote_j.png'...
  -> Entrenando modelos para 'qr_lote_k.png'...
  -> Entrenando modelos para 'qr_lote_b.png'...
  -> E

Saving qr_lote_j_corrupto.png to qr_lote_j_corrupto.png

✅ Imagen corrupta 'qr_lote_j_corrupto.png' lista para ser restaurada.
--------------------------------------------------

🖼️  Iniciando la restauración de 'qr_lote_j_corrupto.png'...
Intentando restaurar: qr_codes_corrupted/qr_lote_j_corrupto.png
  -> SQHN recordó el patrón de: 'qr_lote_j.png'
  -> Autoencoder encontró la mejor coincidencia con: 'qr_lote_j.png'
✅ Visualización guardada en: plots/qr_restoration_result.png
Figure(1200x400)

Copiando la imagen final a Google Drive...
'/content/SQHN/plots/qr_restoration_result.png' -> '/content/drive/MyDrive/SQHN_Project_Final/Restauraciones_QR/qr_restoration_result.png'

✅ ¡ÉXITO! Visualización guardada en: /content/drive/MyDrive/SQHN_Project_Final/Restauraciones_QR


In [ ]:
# --- PASO 1: Preparar el Entorno ---
# Se crean las carpetas para las imágenes limpias y las corruptas.
clean_folder = "qr_codes_clean"
corrupted_folder = "qr_codes_corrupted"
os.makedirs(clean_folder, exist_ok=True)
os.makedirs(corrupted_folder, exist_ok=True)

print("✅ Carpetas 'qr_codes_clean' y 'qr_codes_corrupted' listas.")
print("-" * 60)

# --- PASO 2: Subir las Imágenes de Referencia (Limpias) ---
print("PASO 2: Por favor, sube tus imágenes de QR originales y limpias.")
print("(Puedes seleccionar varios archivos a la vez)")

# Mover los archivos subidos a la carpeta correcta
uploaded_clean = files.upload()
if not uploaded_clean:
    print("\n⚠️ No se subieron archivos limpios. El proceso se ha detenido.")
else:
    for filename, content in uploaded_clean.items():
        shutil.move(filename, os.path.join(clean_folder, filename))
    print(f"\n✅ {len(uploaded_clean)} imágenes limpias movidas a la carpeta '{clean_folder}'.")
    print("-" * 60)

    # --- PASO 3: Construir la Biblioteca de Modelos ---
    print("\n🧠 PASO 3: Construyendo la biblioteca de modelos...")
    print("(Esto puede tardar unos minutos, ya que se entrena un modelo para cada imagen)")
    # Borramos la biblioteca antigua para asegurarnos de que se reconstruye con las nuevas imágenes
    if os.path.exists("qr_model_library.pkl"):
        os.remove("qr_model_library.pkl")
    !python qr_restorer_fair_fight.py learn {clean_folder}
    print("-" * 60)
  # --- PASO 4: Subir la Imagen Corrupta a Restaurar ---
    print("\nPASO 4: Ahora, por favor, sube la imagen de QR corrupta que quieres restaurar.")
    print("(Selecciona solo un archivo)")

    uploaded_corrupted = files.upload()
    if not uploaded_corrupted:
        print("\n⚠️ No se subió ningún archivo corrupto. El proceso se ha detenido.")
    else:
        corrupted_filename = list(uploaded_corrupted.keys())[0]
        # Borrar contenido antiguo de la carpeta de corruptos para evitar confusiones
        for f in os.listdir(corrupted_folder): os.remove(os.path.join(corrupted_folder, f))
        shutil.move(corrupted_filename, os.path.join(corrupted_folder, corrupted_filename))
        image_to_restore_path = os.path.join(corrupted_folder, corrupted_filename)
        print(f"\n✅ Imagen corrupta '{corrupted_filename}' lista para ser restaurada.")
        print("-" * 60)

        # --- PASO 5: Restaurar la Imagen ---
        print(f"\n🖼️  PASO 5: Iniciando la restauración de '{corrupted_filename}'...")
        !python qr_restorer_fair_fight.py restore {image_to_restore_path}

        # --- PASO 6: Guardar el Resultado en Google Drive ---
        project_path_drive = '/content/drive/MyDrive/SQHN_Project_Final'
        destination_drive_plots = os.path.join(project_path_drive, 'Restauraciones_QR')
        os.makedirs(destination_drive_plots, exist_ok=True)

        plots_source_colab = '/content/SQHN/plots'
        output_filename = "qr_restoration_fair_fight_result.png"
        source_file_path = os.path.join(plots_source_colab, output_filename)

        if os.path.exists(source_file_path):
            print("\nPASO 6: Copiando la imagen final a Google Drive...")
            !cp -v {source_file_path} {destination_drive_plots}/
            print(f"\n✅ ¡ÉXITO TOTAL! Visualización guardada en: {destination_drive_plots}")
        else:
            print("\n⚠️ No se encontró el archivo de resultado para copiar a Drive.")

✅ Carpetas 'qr_codes_clean' y 'qr_codes_corrupted' listas.
------------------------------------------------------------
PASO 2: Por favor, sube tus imágenes de QR originales y limpias.
(Puedes seleccionar varios archivos a la vez)


Saving qr_lote_a.png to qr_lote_a.png
Saving qr_lote_b.png to qr_lote_b.png
Saving qr_lote_c.png to qr_lote_c.png
Saving qr_lote_d.png to qr_lote_d.png
Saving qr_lote_e.png to qr_lote_e.png
Saving qr_lote_f.png to qr_lote_f.png
Saving qr_lote_g.png to qr_lote_g.png
Saving qr_lote_h.png to qr_lote_h.png
Saving qr_lote_j.png to qr_lote_j.png
Saving qr_lote_k.png to qr_lote_k.png

✅ 10 imágenes limpias movidas a la carpeta 'qr_codes_clean'.
------------------------------------------------------------

🧠 PASO 3: Construyendo la biblioteca de modelos...
(Esto puede tardar unos minutos, ya que se entrena un modelo para cada imagen)
Construyendo biblioteca de modelos desde: qr_codes_clean
  -> Entrenando modelos para 'qr_lote_d.png'...
  -> Entrenando modelos para 'qr_lote_f.png'...
  -> Entrenando modelos para 'qr_lote_g.png'...
  -> Entrenando modelos para 'qr_lote_j.png'...
  -> Entrenando modelos para 'qr_lote_k.png'...
  -> Entrenando modelos para 'qr_lote_b.png'...
  -> Entrenando model

Saving qr_lote_g_corrupto.png to qr_lote_g_corrupto.png

✅ Imagen corrupta 'qr_lote_g_corrupto.png' lista para ser restaurada.
------------------------------------------------------------

🖼️  PASO 5: Iniciando la restauración de 'qr_lote_g_corrupto.png'...
Intentando restaurar: qr_codes_corrupted/qr_lote_g_corrupto.png
  -> SQHN recordó el patrón de: 'qr_lote_g.png'
  -> Autoencoder (pelea justa) encontró la mejor coincidencia con: 'qr_lote_g.png'

✅ Visualización guardada localmente en: plots/qr_restoration_fair_fight_result.png
Figure(1200x400)

PASO 6: Copiando la imagen final a Google Drive...
'/content/SQHN/plots/qr_restoration_fair_fight_result.png' -> '/content/drive/MyDrive/SQHN_Project_Final/Restauraciones_QR/qr_restoration_fair_fight_result.png'

✅ ¡ÉXITO TOTAL! Visualización guardada en: /content/drive/MyDrive/SQHN_Project_Final/Restauraciones_QR


In [ ]:
# --- PARÁMETRO PRINCIPAL DE LA DEMO ---
# Cambia este número si quieres probar con otra resolución en el futuro (ej. 128).
RESOLUTION = 96
# ------------------------------------

# --- PASO 1: Preparar el Entorno ---
# Crear las carpetas para las imágenes.
clean_folder = "qr_codes_clean"
corrupted_folder = "qr_codes_corrupted"
os.makedirs(clean_folder, exist_ok=True)
os.makedirs(corrupted_folder, exist_ok=True)

print(f"✅ Carpetas listas para la demostración a {RESOLUTION}x{RESOLUTION}px.")
print("-" * 60)

# --- PASO 2: Subir las Imágenes de Referencia (Limpias) ---
print(f"PASO 1: Por favor, sube tus imágenes de QR originales y limpias.")
print("(Puedes seleccionar varios archivos a la vez)")

# Mover los archivos subidos a la carpeta correcta.
uploaded_clean = files.upload()
if not uploaded_clean:
    print("\n⚠️ No se subieron archivos limpios. El proceso se ha detenido.")
else:
    # Limpiar la carpeta de archivos antiguos antes de añadir los nuevos.
    for f in os.listdir(clean_folder): os.remove(os.path.join(clean_folder, f))
    for filename, content in uploaded_clean.items():
        shutil.move(filename, os.path.join(clean_folder, filename))
    print(f"\n✅ {len(uploaded_clean)} imágenes limpias listas en la carpeta '{clean_folder}'.")
    print("-" * 60)

    # --- PASO 3: Construir la Biblioteca de Modelos ---
    print("\n🧠 PASO 2: Construyendo la biblioteca de modelos...")

    # ¡CRUCIAL! Borramos la biblioteca antigua para forzar el reentrenamiento
    # con la nueva resolución.
    library_file = "qr_model_library.pkl"
    if os.path.exists(library_file):
        print(f"   -> Borrando la biblioteca de modelos antigua ('{library_file}')...")
        os.remove(library_file)

    !python qr_restorer_hires.py learn {clean_folder} --resolution {RESOLUTION}
    print("-" * 60)

    # --- PASO 4: Subir la Imagen Corrupta a Restaurar ---
    print(f"\nPASO 3: Ahora, por favor, sube la imagen de QR corrupta que quieres restaurar.")
    print("(Selecciona solo un archivo)")

    uploaded_corrupted = files.upload()
    if not uploaded_corrupted:
        print("\n⚠️ No se subió ningún archivo corrupto. El proceso se ha detenido.")
    else:
        corrupted_filename = list(uploaded_corrupted.keys())[0]
        # Limpiar la carpeta de corruptos para evitar confusiones.
        for f in os.listdir(corrupted_folder): os.remove(os.path.join(corrupted_folder, f))
        shutil.move(corrupted_filename, os.path.join(corrupted_folder, corrupted_filename))
        image_to_restore_path = os.path.join(corrupted_folder, corrupted_filename)
        print(f"\n✅ Imagen corrupta '{corrupted_filename}' lista para ser restaurada.")
        print("-" * 60)

        # --- PASO 5: Restaurar la Imagen ---
        print(f"\n🖼️  PASO 4: Iniciando la restauración a {RESOLUTION}x{RESOLUTION}px...")
        !python qr_restorer_hires.py restore {image_to_restore_path} --resolution {RESOLUTION}

        # --- PASO 6: Guardar el Resultado en Google Drive ---
        project_path_drive = '/content/drive/MyDrive/SQHN_Project_Final'
        destination_drive_plots = os.path.join(project_path_drive, 'Restauraciones_QR_HiRes')
        os.makedirs(destination_drive_plots, exist_ok=True)

        plots_source_colab = '/content/SQHN/plots'
        output_filename = f"qr_restoration_{RESOLUTION}px_result.png"
        source_file_path = os.path.join(plots_source_colab, output_filename)

        if os.path.exists(source_file_path):
            print("\nPASO 5: Copiando la imagen final a Google Drive...")
            !cp -v {source_file_path} {destination_drive_plots}/
            print(f"\n✅ ¡ÉXITO TOTAL! Visualización guardada en: {destination_drive_plots}")
        else:
            print("\n⚠️ No se encontró el archivo de resultado para copiar a Drive.")

✅ Carpetas listas para la demostración a 96x96px.
------------------------------------------------------------
PASO 1: Por favor, sube tus imágenes de QR originales y limpias.
(Puedes seleccionar varios archivos a la vez)


KeyboardInterrupt: 

In [ ]:
# --- PARÁMETRO PRINCIPAL DE LA DEMO ---
# Cambia este número para controlar la calidad. 96 es un buen punto de partida.
# Valores más altos (ej. 128) son más precisos pero más lentos.
RESOLUTION = 96
# ------------------------------------

# --- PASO 1: Preparar el Entorno ---
# Se crean las carpetas para las imágenes limpias y las corruptas.
clean_folder = "qr_codes_clean"
corrupted_folder = "qr_codes_corrupted"
os.makedirs(clean_folder, exist_ok=True)
os.makedirs(corrupted_folder, exist_ok=True)

print(f"✅ Carpetas listas para la demostración a {RESOLUTION}x{RESOLUTION}px.")
print("-" * 60)

# --- PASO 2: Subir las Imágenes de Referencia (Limpias) ---
print(f"PASO 1: Por favor, sube tus imágenes de QR originales y limpias.")
print("(Puedes seleccionar varios archivos a la vez)")

uploaded_clean = files.upload()
if not uploaded_clean:
    print("\n⚠️ No se subieron archivos limpios. El proceso se ha detenido.")
else:
    # Limpiar la carpeta de archivos antiguos para empezar de cero.
    for f in os.listdir(clean_folder): os.remove(os.path.join(clean_folder, f))
    for filename, content in uploaded_clean.items():
        shutil.move(filename, os.path.join(clean_folder, filename))
    print(f"\n✅ {len(uploaded_clean)} imágenes limpias listas en la carpeta '{clean_folder}'.")
    print("-" * 60)

    # --- PASO 3: Construir las Bibliotecas de Modelos ---
    print("\n🧠 PASO 2: Construyendo las bibliotecas de modelos...")

    # ¡CRUCIAL! Borramos los archivos de modelos antiguos para forzar el reentrenamiento
    # con la nueva resolución y el nuevo conjunto de imágenes.
    sqhn_library_file = "sqhn_specialist_library.pkl"
    ae_model_file = "generalist_ae.pth"
    if os.path.exists(sqhn_library_file):
        print(f"   -> Borrando la biblioteca SQHN antigua...")
        os.remove(sqhn_library_file)
    if os.path.exists(ae_model_file):
        print(f"   -> Borrando el modelo Autoencoder antiguo...")
        os.remove(ae_model_file)

    print("\n   Entrenando la biblioteca de especialistas SQHN...")
    !python qr_final_comparison.py learn_sqhn {clean_folder} --resolution {RESOLUTION}

    print("\n   Entrenando el Autoencoder generalista...")
    !python qr_final_comparison.py learn_ae {clean_folder} --resolution {RESOLUTION}
    print("-" * 60)

    # --- PASO 4: Subir la Imagen Corrupta a Restaurar ---
    print(f"\nPASO 3: Ahora, por favor, sube la imagen de QR corrupta que quieres restaurar.")
    print("(Selecciona solo un archivo)")

    uploaded_corrupted = files.upload()
    if not uploaded_corrupted:
        print("\n⚠️ No se subió ningún archivo corrupto. El proceso se ha detenido.")
    else:
        corrupted_filename = list(uploaded_corrupted.keys())[0]
        # Limpiar la carpeta de corruptos para evitar confusiones.
        for f in os.listdir(corrupted_folder): os.remove(os.path.join(corrupted_folder, f))
        shutil.move(corrupted_filename, os.path.join(corrupted_folder, corrupted_filename))
        image_to_restore_path = os.path.join(corrupted_folder, corrupted_filename)
        print(f"\n✅ Imagen corrupta '{corrupted_filename}' lista para ser restaurada.")
        print("-" * 60)

        # --- PASO 5: Restaurar la Imagen ---
        print(f"\n🖼️  PASO 4: Iniciando la restauración a {RESOLUTION}x{RESOLUTION}px...")
        !python qr_final_comparison.py restore {image_to_restore_path} --resolution {RESOLUTION}

        # --- PASO 6: Guardar el Resultado en Google Drive ---
        project_path_drive = '/content/drive/MyDrive/SQHN_Project_Final'
        destination_drive_plots = os.path.join(project_path_drive, 'Restauraciones_QR_Final')
        os.makedirs(destination_drive_plots, exist_ok=True)

        plots_source_colab = '/content/SQHN/plots'
        output_filename = f"final_comparison_{RESOLUTION}px.png"
        source_file_path = os.path.join(plots_source_colab, output_filename)

        if os.path.exists(source_file_path):
            print("\nPASO 5: Copiando la imagen final a Google Drive...")
            !cp -v {source_file_path} {destination_drive_plots}/
            print(f"\n✅ ¡ÉXITO TOTAL! Visualización guardada en: {destination_drive_plots}")
        else:
            print("\n⚠️ No se encontró el archivo de resultado para copiar a Drive.")

✅ Carpetas listas para la demostración a 96x96px.
------------------------------------------------------------
PASO 1: Por favor, sube tus imágenes de QR originales y limpias.
(Puedes seleccionar varios archivos a la vez)


Saving qr_lote_a.png to qr_lote_a.png
Saving qr_lote_b.png to qr_lote_b.png
Saving qr_lote_c.png to qr_lote_c.png
Saving qr_lote_d.png to qr_lote_d.png
Saving qr_lote_e.png to qr_lote_e.png
Saving qr_lote_f.png to qr_lote_f.png
Saving qr_lote_g.png to qr_lote_g.png
Saving qr_lote_h.png to qr_lote_h.png
Saving qr_lote_j.png to qr_lote_j.png
Saving qr_lote_k.png to qr_lote_k.png

✅ 10 imágenes limpias listas en la carpeta 'qr_codes_clean'.
------------------------------------------------------------

🧠 PASO 2: Construyendo las bibliotecas de modelos...

   Entrenando la biblioteca de especialistas SQHN...
Construyendo biblioteca de SQHN especialistas (96x96px)...
  -> Memorizando 'qr_lote_d.png'...
  -> Memorizando 'qr_lote_f.png'...
  -> Memorizando 'qr_lote_g.png'...
  -> Memorizando 'qr_lote_j.png'...
  -> Memorizando 'qr_lote_k.png'...
  -> Memorizando 'qr_lote_b.png'...
  -> Memorizando 'qr_lote_h.png'...
  -> Memorizando 'qr_lote_c.png'...
  -> Memorizando 'qr_lote_e.png'...
  -> M

Saving qr_lote_f_corrupto.png to qr_lote_f_corrupto.png

✅ Imagen corrupta 'qr_lote_f_corrupto.png' lista para ser restaurada.
------------------------------------------------------------

🖼️  PASO 4: Iniciando la restauración a 96x96px...
Restaurando (96x96px): qr_codes_corrupted/qr_lote_f_corrupto.png
  -> SQHN recordó el patrón de: 'qr_lote_f.png'
  -> Autoencoder Generalista está reconstruyendo...
/content/SQHN/qr_final_comparison.py:110: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be load